In [1]:
import os
import numpy as np
import pandas as pd
import shutil
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
# reads excel

filepath = "./04-EPI-074.xlsx"

if os.path.exists(filepath):
    file = pd.ExcelFile(filepath)
else:
    print("the path does not exists")

In [3]:
# compiles all tabs

sheet_names = file.sheet_names

raw_data = pd.DataFrame()

for sheet in sheet_names:
    df = pd.read_excel(filepath, sheet_name=sheet, index_col=0, header=None).dropna(how="all")
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    df.insert(0, "Sheet", sheet)
    
    raw_data = pd.concat([raw_data, df])
    
    del df

In [4]:
# renames & changes types

raw_data.columns = [i.strip() for i in raw_data.columns]

convert_dict = {
    'RT (min)': float, 
    'Base peak (m/z)': float,
    '% Area': float
} 

raw_data = raw_data.astype(convert_dict)

raw_data["Base peak (m/z)"] = raw_data["Base peak (m/z)"].round(0).astype(float)

In [5]:
# filters for area

area_cutoff = 0.1
raw_data_sel = raw_data.loc[raw_data["% Area"] >= area_cutoff]

In [6]:
# finds rt_range based on min_rt & max_rt

rt_interval = 0.05

min_rt = raw_data_sel["RT (min)"].min()
max_rt = raw_data_sel["RT (min)"].max()
rt_range = np.arange(min_rt, max_rt, rt_interval)

In [7]:
%%time
all_data = pd.DataFrame()

for rt in rt_range:
    rt_lower_bound = round(rt, 2)
    rt_upper_bound = round(rt + rt_interval, 2)
    rt_range_text = "{:.2f}-{:.2f}".format(rt_lower_bound, rt_upper_bound)
    
    df = raw_data_sel.loc[(raw_data_sel["RT (min)"] >= rt_lower_bound) & (raw_data_sel["RT (min)"] < rt_upper_bound)].copy()
    df["rt_range"] = rt_range_text
    df.reset_index(drop=True, inplace=True)
    df.set_index("Sheet", inplace=True)
    unique_base_peak = sorted(df["Base peak (m/z)"].unique())

    for base_peak in unique_base_peak:
        df2 = df.loc[df["Base peak (m/z)"] == base_peak, ["% Area"]]
        df2.columns = pd.MultiIndex.from_product([
            ["rt_range: {}".format(rt_range_text)],
            ["Base peak: {}".format(base_peak)]])
                
        df3 = df2.groupby(df2.index).sum()
        
        if all_data.shape[0] == 0:
            all_data = df3
        else:
            all_data = pd.merge(
                left=all_data,
                right=df3,
                left_index=True, 
                right_index=True,
                how = "outer")
        
        del df2, df3

    del df

CPU times: user 11.7 s, sys: 40.9 ms, total: 11.8 s
Wall time: 11.8 s


In [8]:
# reorders indexes

new_indexes = [j[::-1] for j in sorted([i[::-1] for i in all_data.index])]
final_dataset = all_data.loc[new_indexes]

In [9]:
# saves to excel

filename, extension = os.path.splitext(filepath)
dest_filepath = "{}_res{}".format(filename, extension)

with pd.ExcelWriter(dest_filepath) as writer:
    final_dataset.to_excel(
        writer, 
        sheet_name="all_auto", 
        index=True, 
        freeze_panes=(3, 1),        
    )